In [1]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [2]:
# dij需求i到设施j的成本
d = [[1, 2], [1, 2]]
n = 2   # 两个设施点
m = 1   # 两个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]

In [3]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H

def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2
    return Hp
        
# Hp = generate_Hp(n, m, d, g)


In [4]:
import sys
sys.path.append('../')
import z_library.linear_system as ls

In [5]:
# n, m = 1, 1

In [6]:
def gnrt_gate_hdi(u):
    result = []
    nonzero_indices = ls.find_nonzero_indices(u)
    for urow, nrow in zip(u, nonzero_indices):
        # 把非0元素映射成01
        filtered_arr = [0 if x == -1 else 1 for x in urow if x != 0]
        binary_str = ''.join(map(str, filtered_arr))
        # 取到二进制表示的数
        map_num = int(binary_str, 2)
        # print(map_num)
        length = len(nrow)
        scale = 2**length
        matrix = np.zeros((scale, scale))

        matrix[map_num, scale - 1 - map_num] = 1
        map_num = 2**length - 1 - map_num
        # print(map_num)
        matrix[map_num, scale - 1 - map_num] = 1
        result.append(matrix)
    return result

# print("非零元素的索引：", nonzero_indices)

u = ls.gnrt_u(n, m)
u = np.array([[0,  1,  1],
              [1,  1,  1]])
print(u)
nonzero_indices = ls.find_nonzero_indices(u)
gate_hds = gnrt_gate_hdi(u)
print(nonzero_indices)

[[0 1 1]
 [1 1 1]]
[[1, 2], [0, 1, 2]]


In [7]:
from scipy.linalg import expm
def build_circ(params):
  depth = len(params) // 2
  qc = QuantumCircuit(num_qubits - 3)
  beta = params[:depth]
  gamma = params[depth:]

  for dp in range(depth):
    # qc.unitary(expm(-1j * beta[dp] * Hp), range(num_qubits))
    for gate_hdi, ind in zip(gate_hds, nonzero_indices):
      print(gate_hdi, ind)
      qc.unitary(expm(-1j * gamma[dp] * gate_hdi), (num_qubits - 4 - i for i in ind))
  # qc.measure_all()
  return qc

In [8]:
from scipy.linalg import expm
def build_circ1(params):
  depth = len(params) // 2
  qc = QuantumCircuit(num_qubits - 3)
  beta = params[:depth]
  gamma = params[depth:]

  for dp in range(depth):
    # qc.unitary(expm(-1j * beta[dp] * Hp), range(num_qubits))
    for gate_hdi, ind in zip(reversed(gate_hds), reversed(nonzero_indices)):
      print(gate_hdi, ind)
      qc.unitary(expm(-1j * gamma[dp] * gate_hdi), (num_qubits - 4 - i for i in ind))
  # qc.measure_all()
  return qc

In [9]:
from qiskit import transpile
ls.set_print_form()

# 输入qc, 返回电路酉矩阵
def get_circ_unitary(quantum_circuit):
  backend = Aer.get_backend('unitary_simulator' )
  new_circuit = transpile(quantum_circuit, backend)
  job = backend.run(new_circuit)
  result = job.result()
  unitary = result.get_unitary()
  return unitary

m1 = get_circ_unitary(build_circ1([2*np.pi*0.9, 2*np.pi*0.9])) 
m2 = get_circ_unitary(build_circ([2*np.pi*0.9, 2*np.pi*0.9])) 
np.where(m1 != m2)

[[0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]] [0, 1, 2]
[[0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]] [1, 2]
[[0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]] [1, 2]
[[0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]] [0, 1, 2]


(array([0]),)

In [10]:
build_circ([1, 1]).draw()

[[0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]] [1, 2]
[[0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]] [0, 1, 2]


┌──────────┐┌──────────┐
q_0: ┤1         ├┤2         ├
     │  Unitary ││          │
q_1: ┤0         ├┤1 Unitary ├
     └──────────┘│          │
q_2: ────────────┤0         ├
                 └──────────┘

In [11]:
build_circ1([1, 1]).draw()

[[0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]] [0, 1, 2]
[[0. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]] [1, 2]


┌──────────┐┌──────────┐
q_0: ┤2         ├┤1         ├
     │          ││  Unitary │
q_1: ┤1 Unitary ├┤0         ├
     │          │└──────────┘
q_2: ┤0         ├────────────
     └──────────┘

In [12]:
m1

Operator([[ 0.6545+0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.5878j,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.4755j],
          [ 0.    +0.j    ,  1.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ],
          [ 0.    +0.j    ,  0.    +0.j    ,  1.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ],
          [ 0.    +0.4755j,  0.    +0.j    ,  0.    +0.j    ,  0.809 +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    , -0.3455+0.j    ],
          [-0.3455+0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.809 +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.4755j],
          [ 0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  1.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ],
          [ 0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,

In [13]:
m2

Operator([[ 0.6545+0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.4755j, -0.3455+0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.4755j],
          [ 0.    +0.j    ,  1.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ],
          [ 0.    +0.j    ,  0.    +0.j    ,  1.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ],
          [ 0.    +0.5878j,  0.    +0.j    ,  0.    +0.j    ,  0.809 +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ],
          [ 0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.809 +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.5878j],
          [ 0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  1.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ],
          [ 0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,  0.    +0.j    ,